# Параметры игры

In [1]:
agents_num = 10
min_weight = 1 
max_weight = 30
eps = 10 ** (-6)

# Предварительные инициализации

In [2]:
seed()

# Функции для решения исходной задачи

## Генерация исходных данных

### Теорема Биркгофа
Всякая дважды стохастическая матрица представима в виде выпуклой комбинации подстановочных матриц

In [3]:
def generate_random_permutation_matrix(n):
    G = []
    for _ in range(n):
        G.append([0] * n)
    permutation = []
    elems_set = set(range(n))
    while elems_set:
        elem = choice(list(elems_set))
        permutation.append(elem)
        elems_set.remove(elem)

    for i in range(n):
        G[i][permutation[i]] = 1
    return Matrix(G)
        

In [4]:
def generate_stochastic_coefs(n):
    left = 1000
    coefs = [1] * n
    left -= n
    for i in range(n):
        if left != 0:
            coefs[i] += randrange(0, left)
            left -= (coefs[i] - 1)
    coefs [-1] += left
    shuffle(coefs)
    return [coef / 1000 for coef in coefs]

### Теорема Биркгофа
Всякая дважды стохастическая матрица представима в виде выпуклой комбинации подстановочных матриц

In [5]:
def generate_random_stochastic_matrix(n):
    # Birkhoff algorithm
    k = randint(1, n**2 - 2 * n + 2)
    coefs = generate_stochastic_coefs(k)
    permutation_matrixes = []
    for i in range(k):
        while True:
            matrix = generate_random_permutation_matrix(n)
            if matrix not in permutation_matrixes:
                permutation_matrixes.append(matrix)
                break
    G = Matrix(n, n)
    for i in range(k):
        G = G + coefs[i] * permutation_matrixes[i]
    return G

def is_stochastic(G):
    flag = True
    for row in G.rows():
        if sum(row) != 1:
            flag = False
    for col in G.columns():
        if sum(col) != 1:
            flag = False
    return flag

In [6]:
def generate_agents_opinion(agents_num, min_val, max_val):
    return vector([randint(min_val, max_val) for _ in range(agents_num)])

## Моделирование игры

In [7]:
def format_vector_float(vector):
     return "  ".join(["{:.6f}".format(elem) for elem in map(float, vector)])

In [8]:
def stop_func(prev, cur, eps):
    for i in range(len(cur)):
        if prev[i] - cur[i] >= eps:
            return False
    return True

In [9]:
def model_game_without_influence(trust_matrix, agents_opinion, eps):
    prev_state = deepcopy(agents_opinion)
    cur_iter = 0
    while True:
        cur_iter += 1
        cur_state = trust_matrix * prev_state
        print "Current iteration: {}\n{}".format(cur_iter, format_vector_float(cur_state))
        if stop_func(prev_state, cur_state, eps):
            return cur_state
        prev_state = cur_state
    return cur_state

In [10]:
def generate_influence_agents_1_2_players(agents_num):
    k = randint(1, agents_num - 3)
    influence_agents = dict()
    # for the first player
    for i in range(k):
        while True:
            influence_agent = randint(3, agents_num)
            if influence_agent not in influence_agents:
                influence_agents[influence_agent] = 0
                break
    k = randint(1, agents_num - k)
    # for the second player
    for i in range(k):
        while True:
            influence_agent = randint(3, agents_num)
            if influence_agent not in influence_agents:
                influence_agents[influence_agent] = 1
                break
    return {'influence_agents': influence_agents, 'values': [randint(0, 100), randint(-100, 0)]} 

In [11]:
def print_influence_1_2_players(influence):
    influence_agents = influence['influence_agents']
    influence_values = influence['values']
    new_dict = {}
    for key in influence_agents:
        item = influence_agents[key]
        if item not in new_dict:
            new_dict[item] = [key]
        else:
            new_dict[item].append(key)
    for player in new_dict:
        print "For {} player: [{}], value: {}".format(player + 1, new_dict[player], influence_values[player])

In [12]:
def correct_opinion(agents_opinion, influence):
    init_opinion = []
    influence_agents = influence['influence_agents']
    influence_values = influence['values']
    for i in range(len(agents_opinion)):
        if i in influence_agents:
            init_opinion.append(influence_values[influence_agents[i]])
        else:
            init_opinion.append(agents_opinion[i])
    return vector(init_opinion)

In [13]:
def model_game_with_influence(trust_matrix, agents_opinion, influence, eps):
    agents_opinion = correct_opinion(agents_opinion, influence)
    return model_game_without_influence(trust_matrix, agents_opinion, eps)

# Выполнение моделирования игры

##  1. Генерация начального вектора мнений игроков 

In [14]:
agents_opinion = generate_agents_opinion(agents_num, min_weight, max_weight)
print "Initial agents opinion: {}".format(agents_opinion)

Initial agents opinion: (4, 12, 4, 18, 11, 28, 15, 22, 18, 8)


## 2. Генерация матрицы доверия игроков

In [15]:
trust_matrix = generate_random_stochastic_matrix(agents_num)
print(trust_matrix)
if is_stochastic(trust_matrix):
    print "Matrix is stochastic"
else:
    print "Error while generating stochastic matrix"

[699/1000 169/1000    1/500    1/500    1/200    1/500    1/200   3/1000    3/500 107/1000]
[   1/250   7/1000   29/500 141/1000  27/1000    1/250    1/200 687/1000   31/500    1/200]
[ 31/1000    1/200    1/500 689/1000   49/250    1/200    1/200  11/1000   27/500    1/500]
[   1/200    1/200    3/500    1/200    1/500   31/500  57/1000        0  29/1000 829/1000]
[  11/200        0    1/500    1/200   86/125    3/500   19/125     3/50    1/250    7/250]
[  1/1000    8/125  31/1000    7/125   3/1000  207/250   3/1000    1/250    1/250    3/500]
[  18/125   3/1000  61/1000    3/100  61/1000    1/200   86/125   1/1000    1/200    1/500]
[   1/250    1/250 827/1000    1/200   3/1000    1/500  79/1000  61/1000   3/1000    3/250]
[  27/500    1/500    1/200    1/250    1/125   21/250    1/500   29/200   69/100    3/500]
[  3/1000 741/1000    3/500  63/1000   7/1000    1/500    1/250    7/250 143/1000   3/1000]
Matrix is stochastic


## 3. Моделирование игры без влияния

In [16]:
game_without_influence = map(float, model_game_without_influence(trust_matrix, agents_opinion, eps))

Current iteration: 1
6.084000  19.624000  16.195000  9.961000  11.950000  25.374000  12.655000  6.228000  18.460000  13.469000
Current iteration: 2
9.365884  8.483437  10.806874  14.295079  11.538871  23.584934  11.909619  15.229161  16.462203  18.323938
Current iteration: 3
10.300192  14.779400  13.735902  18.057704  11.991396  21.523223  11.612011  11.301057  16.408296  10.290819
Current iteration: 4
11.155674  12.787990  16.409364  11.325339  11.860356  20.457848  11.849266  13.408646  15.675908  15.069609
Current iteration: 5
11.920649  13.403056  11.756042  15.189624  12.075940  19.189479  12.085506  15.781324  15.443419  13.154961
Current iteration: 6
12.358313  15.293364  14.482827  13.529198  12.392468  18.249974  12.196306  12.095702  15.545935  13.855913
Current iteration: 7
13.050653  12.715561  13.391558  14.081658  12.441567  17.576183  12.470592  14.143482  15.044704  15.084043
Current iteration: 8
13.232687  14.096780  13.784135  15.045987  12.706903  16.865631  12.70289

13.999996  13.999998  13.999998  14.000001  13.999987  14.000024  13.999990  13.999997  14.000009  13.999999
Current iteration: 68
13.999997  13.999998  13.999998  14.000001  13.999990  14.000019  13.999992  13.999998  14.000008  14.000000
Current iteration: 69
13.999997  13.999999  13.999999  14.000001  13.999991  14.000016  13.999994  13.999998  14.000006  14.000000
Current iteration: 70
13.999998  13.999999  13.999999  14.000000  13.999993  14.000013  13.999995  13.999998  14.000005  14.000000
Current iteration: 71
13.999998  13.999999  13.999999  14.000000  13.999994  14.000011  13.999996  13.999999  14.000004  14.000000
Current iteration: 72
13.999999  13.999999  13.999999  14.000000  13.999995  14.000009  13.999996  13.999999  14.000003  14.000000
Current iteration: 73
13.999999  13.999999  13.999999  14.000000  13.999996  14.000007  13.999997  13.999999  14.000003  14.000000
Current iteration: 74
13.999999  13.999999  14.000000  14.000000  13.999997  14.000006  13.999998  13.999

In [17]:
print "Game without external influence:\n[{}]".format(format_vector_float(game_without_influence))

Game without external influence:
[13.999999  14.000000  14.000000  14.000000  13.999998  14.000004  13.999998  13.999999  14.000002  14.000000]


## 4. Моделирование игры c влиянием информационных агентов 1 и 2

In [22]:
influence = generate_influence_agents_1_2_players(agents_num)
print_influence_1_2_players(influence)

For 1 player: [[8]], value: 44
For 2 player: [[9, 5, 7]], value: -89


In [28]:
print "Initial agents opinion: {}".format(agents_opinion)
corrected_agents_opinion = correct_opinion(agents_opinion, influence)
print "Corrected initial agents opinion: {}".format(corrected_agents_opinion)

Initial agents opinion: (4, 12, 4, 18, 11, 28, 15, 22, 18, 8)
Corrected initial agents opinion: (4, 12, 4, 18, 11, -89, 15, -89, 44, -89)


In [29]:
game_with_influence = model_game_with_influence(trust_matrix, agents_opinion, influence, eps)

Current iteration: 1
-4.706000  -55.974000  15.599000  -76.952000  1.976000  -72.424000  11.895000  -1.863000  9.895000  13.554000
Current iteration: 2
-11.443240  -11.131622  -52.788890  7.120334  2.427904  -67.224792  5.814289  13.152368  -0.001164  -44.925187
Current iteration: 3
-14.832189  6.455222  4.613513  -41.468687  0.982699  -57.815931  -0.952229  -43.115758  -4.854753  -8.011887
Current iteration: 4
-10.481706  -35.764715  -29.843905  -10.641089  -3.660122  -49.892379  -4.046787  0.645846  -15.431052  0.115908
Current iteration: 5
-13.668485  -4.354696  -9.708175  -4.146228  -4.141745  -45.212941  -7.089567  -25.355068  -15.610742  -29.712353
Current iteration: 6
-13.813457  -20.096335  -5.576269  -28.468939  -7.406019  -38.637658  -8.217078  -10.734451  -19.283305  -6.766488
Current iteration: 7
-14.147286  -13.533036  -23.013714  -9.392706  -8.400062  -35.266624  -9.662637  -6.432050  -19.151697  -20.000188
Current iteration: 8
-14.675922  -8.932893  -10.039836  -20.21310

In [30]:
print "Game with external influence:\n[{}]".format(format_vector_float(game_with_influence))

Game with external influence:
[-15.899999  -15.899999  -15.899999  -15.900000  -15.899996  -15.900007  -15.899997  -15.899999  -15.900003  -15.900000]
